In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import nltk

# Loading our dataset
The data set is provided by yelp as json files. We will use pandas read_json function to read our json files into dataframes. After we've built our dataframes, we will then examine the data frame and find a business that we would like to examine and apply NLP on.

In [2]:
# Load business data

# Mac Directory
# business_df = pd.read_json("../../DataScience/yelp_dataset/business.json", lines=True)

In [3]:
# with open('../../DataScience/yelp_dataset/review.json') as json_file:      
#     data = json_file.readlines()
#     data = list(map(json.loads, data)) 

# review_df = pd.DataFrame(data)

In [4]:
# Load fast text word vector file (MAc)
# file_path = "../../DataScience/wiki-news-300d-1M-subword.vec"


In [5]:
# Windows Directory
business_df = pd.read_json("C:\\Users\\Jack Davis\\Desktop\\yelp_dataset\\business.json", lines=True)

In [6]:
# Windows Directory
with open('C:\\Users\\Jack Davis\\Desktop\\yelp_dataset\\review.json', encoding='utf8') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

review_df = pd.DataFrame(data)

In [7]:
file_path = "C:\\Users\\Jack Davis\\Desktop\\yelp_dataset\\wiki-news-300d-1M-subword.vec"

# Make copies of our dataframes
We do this so we won't accidentaly broadcast any thing and whip our dataframes

In [8]:
# Make copy of the df, so we don't overwrite the orignal data
copy_business = pd.DataFrame.copy(business_df)

In [9]:
copy_review = pd.DataFrame.copy(review_df)

# What is our target business?
We will begin analysis our dataset by using panda's built in fuctions. This will allow us to get a high level view of our data and what it consist of. This will help us determine what types of methods we will use to select the right business to use.

In [10]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4736897 entries, 0 to 4736896
Data columns (total 9 columns):
business_id    object
cool           int64
date           object
funny          int64
review_id      object
stars          int64
text           object
useful         int64
user_id        object
dtypes: int64(4), object(5)
memory usage: 325.3+ MB


In [11]:
copy_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [12]:
copy_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,691 Richmond Rd,"{'RestaurantsPriceRange2': 2, 'BusinessParking...",YDf95gJZaq05wvo7hTQbbQ,"[Shopping, Shopping Centers]",Richmond Heights,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",1,41.541716,-81.493116,Richmond Town Square,,44143,17,2.0,OH
1,2824 Milton Rd,"{'GoodForMeal': {'dessert': False, 'latenight'...",mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...",Charlotte,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",0,35.236870,-80.741976,South Florida Style Chicken & Ribs,Eastland,28215,4,4.5,NC
2,337 Danforth Avenue,"{'BusinessParking': {'garage': False, 'street'...",v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '10:00-19:00', 'Tuesday': '10:00-19...",0,43.677126,-79.353285,The Tea Emporium,Riverdale,M4K 1N7,7,4.5,ON
3,"7702 E Doubletree Ranch Rd, Ste 300",{},CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",Scottsdale,"{'Friday': '9:00-17:00', 'Tuesday': '9:00-17:0...",1,33.565082,-111.916400,TRUmatch,,85258,3,3.0,AZ
4,4719 N 20Th St,"{'RestaurantsTableService': False, 'GoodForMea...",duHFBe87uNSXImQmvBh87Q,"[Sandwiches, Restaurants]",Phoenix,{},0,33.505928,-112.038847,Blimpie,,85016,10,4.5,AZ


In [13]:
# Sort business review
highest_review = business_df.review_count.sort_values(ascending=False)
highest_review.head()

103325    6979
125404    6417
45588     5632
114018    5429
100916    4789
Name: review_count, dtype: int64

In [14]:
target_location = copy_business.iloc[[103325]]

In [15]:
target_location.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
103325,3655 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'HasTV': False, 'Noise...",4JNXUYY8wbaaDmk3BPzlWw,"[French, Steakhouses, Breakfast & Brunch, Rest...",Las Vegas,"{'Monday': '7:00-23:00', 'Tuesday': '7:00-23:0...",1,36.112827,-115.172581,Mon Ami Gabi,The Strip,89109,6979,4.0,NV


In [100]:
target_review = copy_review[copy_review['business_id'] == target_location.business_id.values[0]]
target_review = target_review.reset_index(drop=True)

In [101]:
target_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,4JNXUYY8wbaaDmk3BPzlWw,0,2011-02-22,0,WE9eUYf5EV8AxJjl8QZRtA,5,"Very chic. Although, the menu items doesnt SCR...",0,K6FpHYwcJYznoXXu8ySZHw
1,4JNXUYY8wbaaDmk3BPzlWw,0,2015-04-15,0,7Fl41hKa0wjO3TlvHKD3lw,4,Cute french bistro with great service. I came ...,1,3SGQKsO1J-jcRIp3WNxCeA
2,4JNXUYY8wbaaDmk3BPzlWw,0,2013-11-11,0,Jm4iOfsltS3T59puoV6r8Q,5,"The food is very good, the prices fair and the...",0,LPT8XlpXlHGAp0Ri4Hu4Rw
3,4JNXUYY8wbaaDmk3BPzlWw,3,2015-12-28,2,m6lwvXPCdpKHVp05Bjkhlw,5,One of my favorite go-to brunch spots on the L...,8,3NnPbhmv_vEfPTBp2pnn9Q
4,4JNXUYY8wbaaDmk3BPzlWw,0,2015-09-16,0,nkb40j7kBe2qAci1xpyd3g,5,Simply amazing steak and frites. Got the blue ...,0,xl4rsQqpibUNhR8Jqxp4OQ


# Conclusion
After reviewing our data, sorting, and figuring out which business had the most review, we came to the conclusion of what business we will move forward with. Our selected business is name Mon Ami Gabi and is located in the middle of Las Vegas stripe. This resturant gives our 6979 reviews for us to apply NLP.

In [102]:
nWords = 100000

# Create an matrix filled with 0's (10000 x 300)
embVectors = np.mat(np.zeros((nWords, 300)))
embIndices = {}

In [103]:
# Open the file
file = open(file_path, encoding='utf8')

# Read first line
line = file.readline()

counter = 0

In [104]:
# Iterate through for loops nWords times
for line_pos in range(0, nWords):
    
    # Read the next line in the vector file and split
    line = file.readline()
    line_split = line.split()
    
    # Split out just the word from the vectors
    word = line_split[0].split()
    
    

# Part of Speech Tagging
We will use NLTK and and iterate through our reviews to get a sense of the tagging associated with each word. We first iterate through the df frame series called text. We then pass it through a function called PunktSentenceTokenizer. We can train our data with PunktSentenceTokenizer if we wish or elave it blank to use the pre-train version built in NLTK.

In [105]:
"""
    This function will take in a body of text that has been tokenize into sentences and token it into words.
    Once the sentize token is token into words, we will tag each token with the apporiate POS tagging.
"""

def process_content(text):
    try:
        for i in text:
            words = nltk.tokenize.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            
#             chunkGram = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
            
#             chunkParser = nltk.RegexpParser(chunkGram)
#             chunked = chunkParser.parse(tagged)
            
#             chunked.draw()

            # Classify our name entity as NE
            nameEnt = nltk.ne_chunk(tagged, binary=True)
#           nameEnt.draw()
            
    except Exception as e:
        print(str(e))

In [106]:
# Iterate through the review
for count, review in enumerate(copy_review.text[0:1]):
    
    # Call tokenize to token out the sentence
    cust_text = nltk.tokenize.PunktSentenceTokenizer(review)
    
cust_text = cust_text.tokenize(review)

process_content(cust_text)

# Find out word frequency
We want to tokenize out each review and be able to find out the most common words being used by users. In order to do so, we will iterate through the dataset and call nltk tokenize. This will create a list of words, which we will then append to a list. However, this will create a list of list, which we will then flatten out with list comprehension and make all words lower case. We also want to tweak our algorithm in order to remove common words and puncuations, because nltk recongize those as words and will be irrelevant to our frequency count.

In [163]:
# Create an empty list
all_words = []

# Iterate through the reviews, token them, and append to all word list
for text in target_review.text:
    word = nltk.word_tokenize(text)
    all_words.append(word)

# Flatten out the list of list so it only contains words per element
all_words = [w.lower() for sublist in all_words for w in sublist]

In [164]:
all_words = nltk.FreqDist(all_words)
print(all_words.most_common(10))

[('.', 51969), ('the', 48836), (',', 30171), ('and', 28586), ('a', 21683), ('i', 21442), ('was', 21104), ('to', 16395), ('it', 13816), ('of', 12661)]


# Should we use unsupervised learning to build or models or is it acceptable to use stars and a linear regression since our response variable is continous? 